In [22]:
from langchain_core.prompts import ChatPromptTemplate

from chroma import vector_store_manager
from config import template, model
from process_pdfs import load_tokenized_documents
from retrieval_methods import hybrid_retrieval
import re


In [10]:
tokenized_documents_clean = load_tokenized_documents()

Loaded precomputed tokenized data from tokenized_pdfs_clean.pkl


In [11]:
def answer_question(question, tokenized_documents = tokenized_documents_clean):
    vector_store_manager.update_embedding_model('dicta-il/dictabert')

    retrieved_docs = hybrid_retrieval(question, tokenized_documents)
    docs_content = "\n\n".join(doc[0].page_content for doc in retrieved_docs)

    prompt_template = ChatPromptTemplate.from_template(template)
    prompt = prompt_template.invoke({"question": question, "context": docs_content})

    answer = model.invoke(prompt.messages[0].content)

    return answer, retrieved_docs

In [12]:
questions = [
    "מהי הגישה העיקרית שמתוארת בתחום הנדסת מערכות?",
    "מהם התחומים המרכזיים אותם מכסה המחלקה להנדסה אזרחית וסביבתית?",
    "מהו היקף אפשרויות התעסוקה לבוגרי הנדסה אזרחית וסביבתית?",
    "מהם כיווני הקריירה הפתוחים בפני בוגרי הנדסת ביוטכנולוגיה?",
    "אילו יישומים רפואיים נכללים בתחום ההנדסה הגרעינית?",
    "כיצד משלימים הסטודנטים את התמחותם במסלול חומרים אלקטרוניים?",
    "האם טיוואן זוהי מדינה?",
    "מי ניצח בפרסי הגראמי בשנת 2025?"
]

In [14]:
docs_retrieved = {}
answers = {}

for question in questions:
    answer, doc_retrieved = answer_question(question)
    answers[question] = answer
    docs_retrieved[question] = doc_retrieved

No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started BM25 retrieval
Finished BM25 retrieval
Started dense retrieval
Finished dense retrieval


No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started BM25 retrieval
Finished BM25 retrieval
Started dense retrieval
Finished dense retrieval


No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Started BM25 retrieval
Finished BM25 retrieval
Started dense retrieval
Finished dense retrieval


No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started BM25 retrieval
Finished BM25 retrieval
Started dense retrieval
Finished dense retrieval


No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started BM25 retrieval
Finished BM25 retrieval
Started dense retrieval
Finished dense retrieval


In [23]:
def remove_think_tags(text: str) -> str:
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

In [26]:
def print_nicely(question,answer):
    print(f"Question: \n{question} \n\nAnswer: \n{remove_think_tags(answer)}")

# Simple Questions:

In [32]:
print_nicely(questions[0], answers[questions[0]])

Question: 
מהי הגישה העיקרית שמתוארת בתחום הנדסת מערכות? 

Answer: 
The main approach in Systems Engineering is characterized by an interdisciplinary methodology. This involves integrating knowledge from multiple disciplines such as engineering, computer science, and management to design, develop, and manage complex systems effectively. The focus is on holistic system design, considering all aspects from technical components to human factors and organizational impacts.


In [33]:
print_nicely(questions[1], answers[questions[1]])

Question: 
מהם התחומים המרכזיים אותם מכסה המחלקה להנדסה אזרחית וסביבתית? 

Answer: 
The Civil and Environmental Engineering Department covers several key areas: environmental engineering, building engineering with a focus on structures, data science and industrial engineering specializations, cybersecurity within electrical and computer engineering, and cross-disciplinary work in medical computing through partnerships with health sciences. These fields reflect the department's comprehensive approach to modern engineering challenges.


In [34]:
print_nicely(questions[2], answers[questions[2]])

Question: 
מהו היקף אפשרויות התעסוקה לבוגרי הנדסה אזרחית וסביבתית? 

Answer: 
היקף אפשרויות התעסוקה לבוגרי הנדסה אזרחית וסביבתית כולל תפקידי Projekt מנהל, תכנון בסיסי, פיתוח, ניהול פרויקטים מורכבים, ומשרות מחקר. בוגרים יכולים להתמחות בתחומים כמו אנרגיה גיאותרמית, סולרית, הידרואלקטרית, ואנרגית רוח, או להשתלב בתפקידים בכירים בארגונים ציבוריים ופרטיים. בנוסף, ישנן אפשרויות מגוונות לשילוב במחקר וללימודים מתקדמים לתארים שני ושלישי.


## Anaylsis:

The model is returning its answers sometimes in English instead of Hebrew, even though the prompt and the context are in Hebrew.

In question indexes number 0 and 1, we can see good answers from the model.
In the Hebrew answer though (index number 2), we can see a typo in the English word "Projekt".

# Reasoning Questions + Hallucinations:

In [42]:
print_nicely(questions[3], answers[questions[3]])

Question: 
מהם כיווני הקריירה הפתוחים בפני בוגרי הנדסת ביוטכנולוגיה? 

Answer: 
בוגרי הנדסת ביוטכנולוגיה מצאו כיוונים הקריירה כמו פיתוח טכנולוגיות בביוטכנולוגיה, מחקר והיועץ בתעשיית האנרגיה המתחדשת (גיאותרמית, סולרית וכו'), ainsi כמו משרות בפיתוח תוכנה וחומרה. הם יכולים גם להמשיך לתארים מתקדמים או להיות מעורבים במחקר אקדמי.


In [39]:
print_nicely(questions[4], answers[questions[4]])

Question: 
אילו יישומים רפואיים נכללים בתחום ההנדסה הגרעינית? 

Answer: 
The medical applications within nuclear engineering encompass advanced imaging techniques such as MRI and PET scans, which aid in precise diagnosis. Additionally, it includes radiation therapy for treating conditions like cancer. Nanotechnology plays a role in developing innovative treatments, such as drug delivery systems using nanoparticles. Furthermore, the field contributes to bio-engineering solutions, including prosthetics and robotic-assisted surgeries, enhancing medical capabilities through integrated engineering approaches.


In [40]:
print_nicely(questions[5], answers[questions[5]])

Question: 
כיצד משלימים הסטודנטים את התמחותם במסלול חומרים אלקטרוניים? 

Answer: 
סטודנטים משלימים את התמחותם במסלול חומרים אלקטרוניים על ידי.getSelection של קורסי ה(Core courses) המandratoirs לתחום זה. כמו כן, הם נדרשים להשתתף בפרויקטים סופיים שמעוררים את ידעיהם ומיישמים את מה שלמדו באמצעות כלי תוכנה. הקורסים והפרויקטים הללו מוכנים כדי להביא אותם לידי מכנה משותף לפני ה入り לתואר השני או השלישי במחלקה להנדסת מכונות או חומרים.


## Anaylsis:

We can see that for reasoning questions, the model is having more difficulty answering. For example, in question index 3, we can see some kind of energy industry called "ainsi" which doesn't exist.
In question index 4, we actually see a pretty good answer, the only problem is it should be in Hebrew.
In question index 5, we can see a very wierd answer, comprising both english and hebrew words, and 2 chinese characters

# No context answers:

In [45]:
print_nicely(questions[6], answers[questions[6]])

Question: 
האם טיוואן זוהי מדינה? 

Answer: 
Based on the provided context, there is no information related to whether Taiwan is a country.


In [46]:
print_nicely(questions[7], answers[questions[7]])

Question: 
מי ניצח בפרסי הגראמי בשנת 2025? 

Answer: 
I don't have information on who won the Grammy Awards in 2025 based on the provided context.


## Analysis:

In both cases, where there isn't any relevant context in the pdfs, the model actually does a good job in saying that it doesn't know.

# Trying specific hallucination questions:

In [37]:
hallucination_questions = [
    "Is Taiwan a country?"
    "איזה צבע היה הכובע של דוד בן גוריון כשהכריז על הקמת המדינה?",
    "מה אמר אלברט איינשטיין בהרצאה הסודית שלו ב-1953 על טלפורטציה קוונטית?",
    "איזו שיטת בישול מיוחדת השתמשה בה מלכת אנגליה אליזבת הראשונה להכנת עוף?",
    "מהי ההשפעה הפסיכולוגית של צפייה באור כחול בשעות 3:47 בבוקר לפי מחקר שפורסם בכתב העת המדעי 'NeuroVision' בשנת 2016?",
    "איך קראו למטוס הסודי של חיל האוויר הישראלי שהיה מסוגל לטוס במהירות פי 5 ממהירות הקול בשנות ה-80?"
]


In [44]:
%%capture

docs_retrieved_h = {}
answers_h = {}

for question in hallucination_questions:
    answer, doc_retrieved = answer_question(question)
    answers_h[question] = answer
    docs_retrieved_h[question] = doc_retrieved

Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.

In [50]:
for question in hallucination_questions:
    print_nicely(question, answers_h[question])
    print('\n')

Question: 
איזה צבע היה הכובע של דוד בן גוריון כשהכריז על הקמת המדינה? 

Answer: 
The context does not provide any information related to David Ben-Gurion or the color of his hat during the declaration.


Question: 
מה אמר אלברט איינשטיין בהרצאה הסודית שלו ב-1953 על טלפורטציה קוונטית? 

Answer: 
אין מידע בקשרים על מה אמר אלברט איינשטיין בהרצאה הסודית שלו ב-1953 על טלפורטציה קוונטית.


Question: 
איזו שיטת בישול מיוחדת השתמשה בה מלכת אנגליה אליזבת הראשונה להכנת עוף? 

Answer: 
The context provided does not contain any information about cooking methods used by Queen Elizabeth I of England. Therefore, it is impossible to determine which special method she used based on the given material.

I don't have the necessary information to answer this question accurately from the provided context.


Question: 
מהי ההשפעה הפסיכולוגית של צפייה באור כחול בשעות 3:47 בבוקר לפי מחקר שפורסם בכתב העת המדעי 'NeuroVision' בשנת 2016? 

Answer: 
אני לא יודע.


Question: 
איך קראו למטוס הסודי של חיל האוויר היש

The model knows how to handle questions that he doesnt receive context about quite well